# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
deployment_name =  "gpt-4o-mini" # gpt-4o-mini or gpt-4o

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

# Just for testing: List current AI Foundry Agents

In [3]:
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_6Fi5xvchHAqbT2xpMf4MZNqV', 'object': 'assistant', 'created_at': 1739872033, 'name': 'AnimalPicker', 'description': None, 'model': 'gpt-4o', 'instructions': '**YOUR GOAL**\r\nYou have to do a WEB search with the provided tools, and return **EXCLUSIVELY** the name of an animal.\r\nFor example if the question "what is the most common mammal in Nuova Guinea?", you should do a WEB search and return the kangaroo.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_ImPt29r2ILkK2VqLUUYPiqaa', 'object': 'assistant', 'created_at': 1739832429, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant.', 'tools': [{'type': 'bing_grounding', 'bing_grounding': {'connections': [{'connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub0

# Create the CodeInterpreterTool

In [4]:
from azure.ai.projects.models import CodeInterpreterTool

code_interpreter = CodeInterpreterTool()
print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}


# Create AI Foundry Agent with its CodeInterpreterTool for AI Agents

In [5]:
# ...then, create the AI Foundry Agent, attaching CodeInterpreterTool object
ai_agent = project_client.agents.create_agent(
    model=deployment_name,
    name="my-agent999",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

ai_agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_NdCioL7GhlIkC3ahtOZ9RPgk', 'object': 'assistant', 'created_at': 1739892025, 'name': 'my-agent999', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

# Just for testing: create a new agent and delete it

In [ ]:
# Create a second agent
ai_agent_02 = project_client.agents.create_agent(
    model=deployment_name,
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

# List all agents
print(f"All agents, including the second one: {project_client.agents.list_agents()['data']}\n")

project_client.agents.delete_agent(ai_agent_02.id)

# List all agents
print(f"All agents, after deleting the second one: {project_client.agents.list_agents()['data']}")

# Create the thread and attach a new message to it

In [6]:
# Create a thread
thread = project_client.agents.create_thread()

print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    """,
)
print(f"Messages: {project_client.agents.list_messages(thread_id=thread.id)}")

Created thread: {'id': 'thread_80Efk2dzmqpMzYQwGb7OG1gp', 'object': 'thread', 'created_at': 1739892112, 'metadata': {}, 'tool_resources': {}}

Messages: {'object': 'list', 'data': [{'id': 'msg_oINdOqM3uBR3ekFOGRne0oYl', 'object': 'thread.message', 'created_at': 1739892113, 'assistant_id': None, 'thread_id': 'thread_80Efk2dzmqpMzYQwGb7OG1gp', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Could you please create a bar chart for the operating profit using \n    the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, \n    Company D: $1.8 million\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_oINdOqM3uBR3ekFOGRne0oYl', 'last_id': 'msg_oINdOqM3uBR3ekFOGRne0oYl', 'has_more': False}


# Run the agent syncrhonously

In [ ]:
%%time

# Run the agent
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, assistant_id=ai_agent.id)

print(f"Run status: {run.status}.\n\nRun details: {run}")

# Fetch messages from the thread after the agent run execution

In [ ]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

# Retrieve and download eventual annotations

In [ ]:
# messages.file_path_annotations[0].text.split('/')[-1]
print (f"Nr. of file path annotations: {len(messages.file_path_annotations)}\n")

i=0
for file_path_annotation in messages.file_path_annotations:
    i += 1
    print(f"{i} - File annotation paths: {file_path_annotation}")
    file_name = file_path_annotation.text.split('/')[-1]
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annoation {i} saved as file to: {os.getcwd()}\\{file_name}")

# Retrieve and download eventual images

In [ ]:
print (f"Nr. of image contents: {len(messages.image_contents)}\n")

i=0
# Generate an image file for the bar chart
for image_content in messages.image_contents:
    i += 1
    print(f"{i} - Image content: {image_content}")
    file_name = f"{image_content.image_file.file_id}_image_content.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}\\{file_name}")

# Delete all agents

In [ ]:
print (f"Nr. of existing agents that will be deleted: {len(project_client.agents.list_agents()['data'])}\n")

i = 0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"{i} - Agent {pca.name} ({pca.id}) is not deleted.")

# HIC SUNT LEONES